In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [19]:
folderpath = '/content/gdrive/MyDrive/Community Help /Abuse Image classification/NumPyArrays/'
modelpath='/content/gdrive/MyDrive/Community Help /Abuse Image classification/Models/'

In [ ]:
!pip install opencv-python==4.4.0.44
!pip install opencv-contrib-python==4.4.0.44

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
import joblib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px

In [4]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [5]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report
from skimage.feature import local_binary_pattern,hog
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [6]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
print (cv2 .__version__)

4.8.0


### Features Extration

In [20]:
def get_hog(images, name='hog', save=False):
    result = np.array([hog(img, block_norm='L2') for img in images])
    if save:
        save_feature(result, name)
    return result

#### LBP

In [21]:
def get_lbp(images, name='lbp', save=False):
  result = np.array([local_binary_pattern(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), 10, 3).flatten() for img in images])
  if save:
      save_feature(result, name)

  return result

#### SIFT

In [22]:
def get_sift(images, name='sift', save=False):
    # SIFT descriptor for 1 image
    def get_image_sift(image, vector_size=32):
        alg = cv2.SIFT_create()
        kps = alg.detect(image, None)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # Making descriptor of same size
        # Descriptor vector size is 128
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)

        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
        return dsc
    # SIFT descriptor for all images
    features = []
    for i, img in enumerate(images):
        dsc = get_image_sift(img)
        features.append(dsc)
    result = np.array(features)
    if save:
        save_feature(result, name)
    return result

#### Color Histogram

In [23]:
def get_color_hist(images, name='color_hist', save=False):
    histograms = []
    for img in images:
        histograms.append(cv2.calcHist([img], [0, 1, 2],None, [8,8,8], [0, 256, 0, 256, 0, 256]).flatten())
    result = np.array(histograms)
    if save:
        save_feature(result, name)
    return result

#### KAZE

In [24]:
def get_kaze(images, name='kaze', save=False):
    def get_image_kaze(image, vector_size=32):
        alg = cv2.KAZE_create()
        kps = alg.detect(image)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)
        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
        return dsc
    # KAZE descriptor for all images
    features = []
    for i, img in enumerate(images):
        dsc = get_image_kaze(img)
        features.append(dsc)
    result = np.array(features)
    if save:
        save_feature(result, name)
    return result

## Normalize Features

In [25]:
 def norm_features_minmax(train, test):
      min_max_scaler = preprocessing.MinMaxScaler()
      norm_train = min_max_scaler.fit_transform(train)
      norm_test = min_max_scaler.transform(test)
      return norm_train, norm_test

In [26]:
def norm_features_zscore(train, test):
    min_max_scaler = preprocessing.StandardScaler()
    norm_train = min_max_scaler.fit_transform(train)
    norm_test = min_max_scaler.transform(test)
    return norm_train, norm_test

In [27]:
def norm_features_minmax2(train,val,test):
    min_max_scaler = preprocessing.MinMaxScaler()
    norm_train = min_max_scaler.fit_transform(train)
    norm_val = min_max_scaler.transform(val)
    norm_test = min_max_scaler.transform(test)
    return norm_train,norm_val, norm_test

In [28]:
def norm_features_zscore2(train,val, test):
    min_max_scaler = preprocessing.StandardScaler()
    norm_train = min_max_scaler.fit_transform(train)
    norm_val = min_max_scaler.transform(val)
    norm_test = min_max_scaler.transform(test)
    return norm_train,norm_val,norm_test

In [29]:
def flattened(images, color=cv2.COLOR_RGB2GRAY, name='flattened', save=False):
    """
    color: default RGB2GRAY, if None is passed then color is used as it is.
    """
    color_images = []
    if color is not None:
        for img in images:
            color_images.append(cv2.cvtColor(img, color))
    else:
        color_images = images

    count = len(color_images)

    result = np.array(color_images).reshape(count, -1)

    if save:
        save_feature(result, name)

    return result

In [30]:
def combine_features(features, horizontal=True):
    if horizontal:
        return np.hstack(features)
    else:
        return np.vstack(features)

In [32]:
def save_feature_nmpy(feature, name):
  np.save(folderpath+ name  +'.npy',feature)
def load_feature_nmpy(feature,name):
  np.load(folderpath+ name  +'.npy')

## Load Data

In [33]:
train_imgs = np.load(folderpath+'train_imgs_color.npy')
train_y = np.load(folderpath+'train_y_color.npy')
val_imgs = np.load(folderpath+'val_imgs_color.npy')
val_y = np.load(folderpath+'val_y_color.npy')

X = np.load(folderpath+'X_color.npy')
y = np.load(folderpath+'y_color.npy')
X_test = np.load(folderpath+'X_test_color.npy')
y_test = np.load(folderpath+'y_test_color.npy')

In [34]:
X.shape  , X_test.shape , np.array(y).shape ,  np.array(y_test).shape

((84844, 224, 224, 3), (21212, 224, 224, 3), (84844,), (21212,))

In [35]:
train_imgs.shape , val_imgs.shape , np.array(train_y).shape , np.array(val_y).shape

((76359, 224, 224, 3), (8485, 224, 224, 3), (76359,), (8485,))

## Training

### SIFT Features

In [ ]:
sift_train = get_sift(train_imgs, name='sift_train', save=False)

In [ ]:
save_feature_nmpy(sift_train, "sift_train")

In [ ]:
sift_val = get_sift(val_imgs, name='sift_val', save=False)

In [ ]:
save_feature_nmpy(sift_val, "sift_val")

In [ ]:
sift_test = get_sift(X_test, name='sift_test', save=False)

In [ ]:
save_feature_nmpy(sift_test, "sift_test")

In [ ]:
sift_train = np.load(folderpath+'sift_train.npy')

In [ ]:
sift_val = np.load(folderpath+'sift_val.npy')


In [ ]:
sift_test = np.load(folderpath+'sift_test.npy')

In [ ]:
sift_train.shape,sift_val.shape,sift_test.shape

((16145, 4096), (1794, 4096), (4485, 4096))

### Gray

### Color Histogram

In [ ]:
hist_train = get_color_hist(train_imgs, name='hist_train', save=False)
hist_val = get_color_hist(val_imgs, name='hist_val', save=False)
hist_test = get_color_hist(X_test, name='hist_val', save=False)

In [ ]:
save_feature_nmpy(hist_train,'hist_train')
save_feature_nmpy(hist_val,'hist_val')
save_feature_nmpy(hist_test,'hist_test')

In [ ]:
hist_train = np.load(folderpath+'hist_train.npy')
hist_val = np.load(folderpath+'hist_val.npy')
hist_test = np.load(folderpath+'hist_test.npy')

In [ ]:
hist_train.shape, hist_val.shape, hist_test.shape

((16145, 512), (1794, 512), (4485, 512))

### KAZE

In [ ]:
kaze_train = get_kaze(train_imgs, name='kaze_train', save=False)
kaze_val = get_kaze(val_imgs, name='kaze_val', save=False)
kaze_test = get_kaze(X_test, name='kaze_test', save=False)

In [ ]:
np.save(folderpath+'kaze_test.npy', kaze_test)
np.save(folderpath+'kaze_train.npy', kaze_train)
np.save(folderpath+'kaze_val.npy', kaze_val)

In [ ]:
kaze_train = np.load(folderpath+'kaze_train.npy')
kaze_val = np.load(folderpath+'kaze_val.npy')
kaze_test = np.load(folderpath+'kaze_test.npy')

In [ ]:
kaze_train.shape, kaze_val.shape, kaze_test.shape

((16145, 2048), (1794, 2048), (4485, 2048))

### HOG

In [ ]:
hog_train = get_hog(train_imgs, name='hog_train', save=False)

In [ ]:
save_feature_nmpy(hog_train, "hog_train")

In [ ]:
hog_val = get_hog(val_imgs, name='hog_val', save=False)

In [ ]:
save_feature_nmpy(hog_val, "hog_val")

In [ ]:
hog_test = get_hog(X_test, name='hog_test', save=False)

In [ ]:
save_feature_nmpy(hog_test, "hog_test")

In [ ]:
hog_train = np.load(folderpath+'hog_train.npy')
hog_val = np.load(folderpath+'hog_val.npy')
hog_test = np.load(folderpath+'hog_test.npy')

In [ ]:
hog_train.shape,hog_val.shape,hog_test.shape

((16145, 54756), (1794, 54756), (4485, 54756))

### LBP

In [ ]:
lbp_train = get_lbp(train_imgs, name='lbp_train', save=False)

In [ ]:
save_feature_nmpy(lbp_train,'lbp_train')

In [ ]:
lbp_val = get_lbp(val_imgs, name='lbp_val', save=False)

In [ ]:
save_feature_nmpy(lbp_val,'lbp_val')

In [ ]:
lbp_test = get_lbp(X_test, name='lbp_test', save=False)

In [ ]:
save_feature_nmpy(lbp_test,'lbp_test')

In [ ]:
lbp_train = np.load(folderpath+'lbp_train.npy')
lbp_val = np.load(folderpath+'lbp_val.npy')
lbp_test = np.load(folderpath+'lbp_test.npy')

In [ ]:
lbp_train.shape,lbp_val.shape,lbp_test.shape

((16145, 50176), (1794, 50176), (4485, 50176))

## Reduce Features by PCA

### HOG

In [ ]:
norm_hog_train, norm_hog_val,norm_hog_test = norm_features_minmax2(hog_train, hog_val,hog_test)

In [ ]:
np.save(folderpath+'norm_hog_train.npy', norm_hog_train)
np.save(folderpath+'norm_hog_val.npy', norm_hog_val)
np.save(folderpath+'norm_hog_test.npy', norm_hog_test)

In [ ]:
norm_hog_train=np.load(folderpath+'norm_hog_train.npy')

In [ ]:
pca = PCA(n_components=100)
pca_hog_train = pca.fit_transform(norm_hog_train)

In [ ]:
np.save(folderpath+'pca_hog_train.npy', pca_hog_train)

In [ ]:
norm_hog_train=''
pca_hog_train=''

In [ ]:
norm_hog_val=np.load(folderpath+'norm_hog_val.npy')

In [ ]:
pca_hog_val = pca.fit_transform(norm_hog_val)

In [ ]:
np.save(folderpath+'pca_hog_val.npy', pca_hog_val)

In [ ]:
norm_hog_val=''
pca_hog_val=''

In [ ]:
norm_hog_test=np.load(folderpath+'norm_hog_test.npy')

In [ ]:
pca_hog_test = pca.fit_transform(norm_hog_test)

In [ ]:
np.save(folderpath+'pca_hog_test.npy', pca_hog_test)

In [ ]:
norm_hog_test=''
pca_hog_test=''

### LBP

In [ ]:
norm_lbp_train, norm_lbp_val,norm_lbp_test = norm_features_minmax2(lbp_train, lbp_val,lbp_test)

In [ ]:
np.save(folderpath+'norm_lbp_train.npy', norm_lbp_train)
np.save(folderpath+'norm_lbp_val.npy', norm_lbp_val)
np.save(folderpath+'norm_lbp_test.npy', norm_lbp_test)

In [ ]:
norm_lbp_train=''
norm_lbp_val=''
norm_lbp_test=''

In [ ]:
norm_lbp_train = np.load(folderpath+'norm_lbp_train.npy')

In [ ]:
pca = PCA(n_components=100)
pca_lbp_train = pca.fit_transform(norm_lbp_train)

In [ ]:
np.save(folderpath+'pca_lbp_train.npy', pca_lbp_train)

In [ ]:
pca_lbp_train=''
norm_lbp_train=''

In [ ]:
norm_lbp_val = np.load(folderpath+'norm_lbp_val.npy')

In [ ]:
pca_lbp_val = pca.fit_transform(norm_lbp_val)

In [ ]:
np.save(folderpath+'pca_lbp_val.npy', pca_lbp_val)

In [ ]:
pca_lbp_val=''
norm_lbp_val=''

In [ ]:
norm_lbp_test = np.load(folderpath+'norm_lbp_test.npy')

In [ ]:
pca_lbp_test = pca.fit_transform(norm_lbp_test)

In [ ]:
np.save(folderpath+'pca_lbp_test.npy', pca_lbp_test)

In [ ]:
pca_lbp_test=''
norm_lbp_test=''

In [ ]:
pca_lbp_train = np.load(folderpath+'pca_lbp_train.npy')
pca_lbp_val = np.load(folderpath+'pca_lbp_val.npy')
pca_lbp_test = np.load(folderpath+'pca_lbp_test.npy')

In [ ]:
print(np.shape(pca_lbp_train),np.shape(pca_lbp_val))

### SIFT

In [ ]:
norm_sift_train, norm_sift_val,norm_sift_test = norm_features_minmax2(sift_train, sift_val,sift_test)

In [ ]:
np.save(folderpath+'norm_sift_train.npy', norm_sift_train)
np.save(folderpath+'norm_sift_val.npy', norm_sift_val)
np.save(folderpath+'norm_sift_test.npy', norm_sift_test)

In [ ]:
pca = PCA(n_components=100)
pca_sift_train = pca.fit_transform(norm_sift_train)
pca_sift_val = pca.transform(norm_sift_val)
pca_sift_test = pca.transform(norm_sift_test)

In [ ]:
np.save(folderpath+'pca_sift_train.npy', pca_sift_train)
np.save(folderpath+'pca_sift_val.npy', pca_sift_val)
np.save(folderpath+'pca_sift_test.npy', pca_sift_test)

In [ ]:
pca_sift_train = np.load(folderpath+'pca_sift_train.npy')
pca_sift_val = np.load(folderpath+'pca_sift_val.npy')
pca_sift_test = np.load(folderpath+'pca_sift_test.npy')

### Gray

In [ ]:
norm_gray_train, norm_gray_val,norm_gray_test = norm_features_minmax2(flat_gray_train, flat_gray_val,flat_gray_test)

In [ ]:
pca = PCA(n_components=100)
pca_gray_train = pca.fit_transform(norm_gray_train)
pca_gray_val = pca.transform(norm_gray_val)
pca_gray_test = pca.transform(norm_gray_test)

In [ ]:
np.save(folderpath+'pca_gray_train.npy', pca_gray_train)
np.save(folderpath+'pca_gray_val.npy', pca_gray_val)
np.save(folderpath+'pca_gray_test.npy', pca_gray_test)

In [ ]:
pca_gray_train=''
pca_gray_val=''
pca_gray_test=''

In [ ]:
pca_gray_train = np.load(folderpath+'pca_gray_train.npy')
pca_gray_val = np.load(folderpath+'pca_gray_val.npy')
pca_gray_test = np.load(folderpath+'pca_gray_test.npy')

### KAZE

In [ ]:
norm_kaze_train, norm_kaze_val, norm_kaze_test = norm_features_minmax2(kaze_train, kaze_val,kaze_test)

In [ ]:
np.save(folderpath+'norm_kaze_train.npy', norm_kaze_train)
np.save(folderpath+'norm_kaze_val.npy', norm_kaze_val)
np.save(folderpath+'norm_kaze_test.npy', norm_kaze_test)

In [ ]:
norm_kaze_train.shape, norm_kaze_val.shape, norm_kaze_test.shape

In [ ]:
pca = PCA(n_components=100)
pca_kaze_train = pca.fit_transform(norm_kaze_train)
pca_kaze_val = pca.transform(norm_kaze_val)
pca_kaze_test = pca.transform(norm_kaze_test)

In [ ]:
np.save(folderpath+'pca_kaze_train.npy', pca_kaze_train)
np.save(folderpath+'pca_kaze_val.npy', pca_kaze_val)
np.save(folderpath+'pca_kaze_test.npy', pca_kaze_test)

In [ ]:
pca_kaze_train =np.load(folderpath+'pca_kaze_train.npy')
pca_kaze_val = np.load(folderpath+'pca_kaze_val.npy')
pca_kaze_test = np.load(folderpath+'pca_kaze_test.npy')

In [ ]:
pca_kaze_train.shape, pca_kaze_val.shape, pca_kaze_test.shape

### Color Histogram

In [ ]:
norm_hist_train, norm_hist_val, norm_hist_test = norm_features_minmax2(hist_train, hist_val, hist_test)

In [ ]:
np.save(folderpath+'norm_hist_train.npy', norm_hist_train)
np.save(folderpath+'norm_hist_val.npy', norm_hist_val)
np.save(folderpath+'norm_hist_test.npy', norm_hist_test)

In [ ]:
from sklearn.decomposition import PCA
pca_1 = PCA(n_components=100)
pca_color_hist_train = pca_1.fit_transform(norm_hist_train)
pca_color_hist_val = pca_1.transform(norm_hist_val)
pca_color_hist_test = pca_1.transform(norm_hist_test)

In [ ]:
np.save(folderpath+'pca_color_hist_train.npy', pca_color_hist_train)
np.save(folderpath+'pca_color_hist_val.npy', pca_color_hist_val)
np.save(folderpath+'pca_color_hist_test.npy', pca_color_hist_test)

In [ ]:
pca_color_hist_train = np.load(folderpath+'pca_color_hist_train.npy')
pca_color_hist_val = np.load(folderpath+'pca_color_hist_val.npy')
pca_color_hist_test = np.load(folderpath+'pca_color_hist_test.npy')

## Reduce Features by LDA

### HOG

In [ ]:
norm_hog_train=np.load(folderpath+'norm_hog_train.npy')

In [ ]:
lda = LDA()
lda_pca_train = lda.fit_transform(norm_hog_train, train_y)

In [ ]:
norm_hog_train=''

In [ ]:
norm_hog_val=np.load(folderpath+'norm_hog_val.npy')

In [ ]:
lda_pca_val = lda.transform(norm_hog_val)

In [ ]:
norm_hog_val=''

In [ ]:
norm_hog_test=np.load(folderpath+'norm_hog_test.npy')

In [ ]:
lda_pca_test = lda.transform(norm_hog_test)

In [ ]:
norm_hog_test=''

### SIFT

In [ ]:
norm_surf_train=np.load(folderpath+'norm_sift_train.npy')
norm_surf_val=np.load(folderpath+'norm_sift_val.npy')
norm_surf_test=np.load(folderpath+'norm_sift_test.npy')

In [ ]:
lda = LDA()
lda_sift_train = lda.fit_transform(norm_surf_train, train_y)
lda_sift_val = lda.transform(norm_surf_val)
lda_sift_test = lda.transform(norm_surf_test)

In [ ]:
np.save(folderpath+'lda_sift_train.npy', lda_sift_train)
np.save(folderpath+'lda_sift_val.npy', lda_sift_val)
np.save(folderpath+'lda_sift_test.npy', lda_sift_test)

In [ ]:
lda_sift_train = np.load(folderpath+'lda_sift_train.npy')
lda_sift_val = np.load(folderpath+'lda_sift_val.npy')
lda_sift_test = np.load(folderpath+'lda_sift_test.npy')

### Color Histogram

In [ ]:
norm_hist_train=np.load(folderpath+'norm_hist_train.npy' )
norm_hist_val=np.load(folderpath+'norm_hist_val.npy' )
norm_hist_test=np.load(folderpath+'norm_hist_test.npy' )

In [ ]:
lda = LDA()
lda_hist_train = lda.fit_transform(norm_hist_train, train_y)
lda_hist_val = lda.transform(norm_hist_val)
lda_hist_test = lda.transform(norm_hist_test)

In [ ]:
np.save(folderpath+'lda_hist_train.npy', lda_hist_train)
np.save(folderpath+'lda_hist_val.npy', lda_hist_val)
np.save(folderpath+'lda_hist_test.npy', lda_hist_test)

In [ ]:
lda_hist_train = np.load(folderpath+'lda_hist_train.npy')
lda_hist_val = np.load(folderpath+'lda_hist_val.npy')
lda_hist_test =  np.load(folderpath+'lda_hist_test.npy')

In [ ]:
lda_hist_train.shape, lda_hist_val.shape, lda_hist_test.shape

### KAZE

In [ ]:
norm_kaze_train=np.load(folderpath+'norm_kaze_train.npy' )
norm_kaze_val=np.load(folderpath+'norm_kaze_val.npy')
norm_kaze_test=np.load(folderpath+'norm_kaze_test.npy' )

In [ ]:
lda = LDA()
lda_kaze_train = lda.fit_transform(norm_kaze_train,train_y)
lda_kaze_val = lda.transform(norm_kaze_val)
lda_kaze_test = lda.transform(norm_kaze_test)

In [ ]:
np.save(folderpath+'lda_kaze_train.npy', lda_kaze_train)
np.save(folderpath+'lda_kaze_val.npy', lda_kaze_val)
np.save(folderpath+'lda_kaze_test.npy', lda_kaze_test)

In [ ]:
lda_kaze_train = np.load(folderpath+'lda_kaze_train.npy')
lda_kaze_val = np.load(folderpath+'lda_kaze_val.npy')
lda_kaze_test = np.load(folderpath+'lda_kaze_test.npy')

In [ ]:
lda_kaze_train.shape, lda_kaze_val.shape, lda_kaze_test.shape

### Combine Features

In [ ]:
hog_train = np.load(folderpath+'hog_train.npy')
hog_val = np.load(folderpath+'hog_val.npy')

kaze_train = np.load(folderpath+'kaze_train.npy')
kaze_val = np.load(folderpath+'kaze_val.npy')

hist_train = np.load(folderpath+'hist_train.npy')
hist_val = np.load(folderpath+'hist_val.npy')

sift_train = np.load(folderpath+'sift_train.npy')
sift_val = np.load(folderpath+'sift_val.npy')

In [ ]:
features_train = None
features_val = None

for t, v in zip([hog_train, hist_train, sift_train, kaze_train],
                   [hog_val, hist_val, sift_val, kaze_val]):
  if features_train is None:
    features_train = t
    features_val = v
  else:
    features_train = combine_features([features_train, t])
    features_val = combine_features([features_val, v])

In [ ]:
np.save(folderpath+'features_train.npy', features_train)
np.save(folderpath+'features_val.npy', features_val)


## Combine Features for PCA  

In [ ]:
pca_hog_train = np.load(folderpath+'pca_hog_train.npy')
pca_hog_val = np.load(folderpath+'pca_hog_val.npy')

pca_kaze_train = np.load(folderpath+'pca_kaze_train.npy')
pca_kaze_val = np.load(folderpath+'pca_kaze_val.npy')

pca_color_hist_train = np.load(folderpath+'pca_color_hist_train.npy')
pca_color_hist_val = np.load(folderpath+'pca_color_hist_val.npy')

pca_sift_train = np.load(folderpath+'pca_sift_train.npy')
pca_sift_val = np.load(folderpath+'pca_sift_val.npy')

pca_lbp_train = np.load(folderpath+'pca_lbp_train.npy')
pca_lbp_val = np.load(folderpath+'pca_lbp_val.npy')

In [ ]:
features_train_pca = None
features_val_pca = None

for t, v in zip([pca_hog_train, pca_color_hist_train, pca_sift_train,  pca_lbp_train, pca_kaze_train],
                   [pca_hog_val, pca_color_hist_val, pca_sift_val, pca_lbp_val, pca_kaze_val]):
  if features_train_pca is None:
    features_train_pca = t
    features_val_pca = v
  else:
    features_train_pca = combine_features([features_train_pca, t])
    features_val_pca = combine_features([features_val_pca, v])

In [ ]:
np.save(folderpath+'features_train_pca.npy', features_train_pca)
np.save(folderpath+'features_val_pca.npy', features_val_pca)

In [ ]:
features_train_pca = np.load(folderpath+'features_train_pca.npy')
features_val_pca = np.load(folderpath+'features_val_pca.npy')

## Combine Features for LDA

In [ ]:
lda_kaze_train = np.load(folderpath+'lda_kaze_train.npy')
lda_kaze_val = np.load(folderpath+'lda_kaze_val.npy')

lda_hist_train = np.load(folderpath+'lda_hist_train.npy')
lda_hist_val = np.load(folderpath+'lda_hist_val.npy')

lda_sift_train = np.load(folderpath+'lda_sift_train.npy')
lda_sift_val = np.load(folderpath+'lda_sift_val.npy')

In [ ]:
features_train_lda = None
features_val_lda = None

for t, v in zip([lda_hist_train, lda_sift_train, lda_kaze_train],
                   [lda_hist_val, lda_sift_val, lda_kaze_val]):
  if features_train_lda is None:
    features_train_lda = t
    features_val_lda = v
  else:
    features_train_lda = combine_features([features_train_lda, t])
    features_val_lda = combine_features([features_val_lda, v])

In [ ]:
np.save(folderpath+'features_train_lda.npy', features_train_lda)
np.save(folderpath+'features_val_lda.npy', features_val_lda)

In [ ]:
features_train_lda = np.load(folderpath+'features_train_lda.npy')
features_val_lda = np.load(folderpath+'features_val_lda.npy')

## Reduce Features by LDA on PCA

In [ ]:
lda = LDA()
features_train_pca = np.load(folderpath+'features_train_pca.npy')
features_val_pca = np.load(folderpath+'features_val_pca.npy')
lda_pca_train = lda.fit_transform(features_train_pca, train_y)
lda_pca_val = lda.transform(features_val_pca)
np.save(folderpath+'lda_pca_train.npy', lda_pca_train)
np.save(folderpath+'lda_pca_val.npy', lda_pca_val)

In [ ]:
lda = LDA()
features_train_pca = np.load(folderpath+'features_train_pca.npy')
features_test_pca = np.load(folderpath+'features_test_pca.npy')
lda_pca_train = lda.fit_transform(features_train_pca, train_y)
lda_pca_test = lda.transform(features_test_pca)
np.save(folderpath+'lda_pca_test.npy', lda_pca_test)